# Code template for hand-in on reconstruction attacks.

Advanced Topics in Machine Learning, U. Copenhagen, fall 2024

Created by Rasmus Pagh with minor edits by Thomas Christensen

Queries on a hidden dataset x from {-1,+1}^100 can be made via the query method below
which calls a web API that allows dot product queries with vectors in {-1,+1}^100.
To protect data, Laplace noise is added to responses. Using the techniques you have
seen in the lecture it is possible to partially reconstruct the dataset using 200 queries.
To make sure that you get a unique challenge, choose any unique string as your challenge
identifier. The web API will keep track of the number of queries made for each identifier.

# Support function for querying the web API

In [1]:
import numpy as np
import requests as rq

# Retrieve answer to challenge for a given query
def query(challenge_id, query_vector, submit=False):
    # Only alphanumeric challenge_id and vextor entries in {-1,+1} are allowed:
    assert(challenge_id.isalnum())
    assert(np.max(np.minimum(np.abs(query_vector-1),np.abs(query_vector+1)))==0)

    # if query array is 1d, make it 2d
    if query_vector.ndim == 1:
        query_vector = query_vector.reshape(1,-1)

    payload = { 'challengeid': challenge_id, 'submit': submit,
                'query': str(query_vector.tolist()) }
    response = rq.post("https://baconbreaker.pythonanywhere.com/query", data = payload).json()
    if submit == False:
        return np.array(eval(response['result']))
    else:
        return response['result']

# Making random queries to the API

In [28]:
challenge_id = 'ThisWasNotTaken123454321' # identifier for hidden dataset
n = 100 # number of entries in hidden dataset
num_queries = 2*n # number of queries to be asked

queries = np.random.choice([-1,+1], size=(num_queries,n)) # Set of random queries
query_results = query(challenge_id, queries)

print(query_results)

[  1. -13.  -7. -39. -17.   7.  17.  36. -18.  29. -50. -32.  -9.  -9.
 -20.  35.   6.  11. -27.  16.   4.  -7. -17. -23.  19. -11. -20.  19.
 -35.  14.  27.  10. -14.   6.  15. -10.  24.  -2.   4.  47.   2. -11.
  36. -16.  -2.  -5.  -4.   6.   2. -10. -26.  35.   1.  -3.  -1.   1.
  -6.   8. -14.  -7.  11. -10.  52. -16.  -8.  -1. -18.  18.   3.  10.
  11. -23. -10. -12.   5.  21.  14.  -2.  21.   1.  -5.  -3.  48.  -8.
   5.   4.  -3.   9. -35.  42.  -1.   1.  -3. -16. -12.  26.  32.   9.
   5. -25. -48.  -9. -10. -26.  44.  31.  -5.  -7. -10. -18.  11.   9.
   9.  -8. -18. -10. -11.  38.  -4.  36. -31.  -7.  18.   9.   2.   4.
 -13.  53. -11.  -6. -20. -20. -38. -23.   9. -14.  -4.  -6.  -4.   2.
  12. -20.  22.   2. -22.  -1.  -2.   7.  -6.  -2.  -5.  19. -16. -33.
  18.  11.  -6.   2.   5. -25.   0. -17.  -8.   7.  22.  17. -25.  37.
  -2.  -8.   0.  10.   8.   8.  -4.  22.  -1. -12. -23. -10. -22.  -7.
  21. -16.  28.  12. -15.  27.  16.  11.  12.   8. -11.  10.   0.  -8.
 -17. 

# Baseline reconstruction attack: Guess equals the best query

Copy the notebook and replace with your attack. The first submission for a given challenge_id is stored on the server and on the leaderboard: http://baconbreaker.pythonanywhere.com/leaderboard/?n=100

**Tip**: to solve a linear program you can use the following import:

```
from scipy.optimize import linprog
```



In [25]:
best_query_number = np.argmax(query_results)
best_query = queries[best_query_number]
best_query_result = query(challenge_id, best_query, submit=True)
print(f"\nReconstruction attack achieves fraction {(1 + best_query_result / n) / 2} correct values")


Reconstruction attack achieves fraction 0.48 correct values


In [129]:
    import math
    from scipy.optimize import linprog
    # Number of people in dataset
    n = 100
    t = 2 * n  # number of queries

    queries = np.random.choice([-1, +1], size=(t, n))
    noisy_results = query(challenge_id, queries)

    # Objective function coefficients: minimize sum of epsilons
    c = np.concatenate([np.zeros(n), np.ones(t)])

    # Inequality constraints
    A_ub = np.zeros((2 * t, n + t))  # 2 constraints per query
    b_ub = np.zeros(2 * t)
    for i in range(t):
        A_ub[2 * i, :n] = queries[i]       # ⟨hat_x, q_i⟩ term
        A_ub[2 * i, n + i] = -1            # -epsilon_i
        b_ub[2 * i] = noisy_results[i]     # upper bound

        A_ub[2 * i + 1, :n] = -queries[i]  # -⟨hat_x, q_i⟩ term
        A_ub[2 * i + 1, n + i] = -1        # -epsilon_i
        b_ub[2 * i + 1] = -noisy_results[i]# lower bound

    # Bounds for the decision variables
    # hat_x is bounded between -1 and 1, epsilons are >= 0
    bounds = [(-1, 1)] * n + [(0, None)] * t

    # Solve the linear program
    result = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=bounds, method='highs')

    # Extract the reconstructed x
    reconstructed_x = result.x[:n]
    print("Reconstructed x:", np.sign(reconstructed_x))

    # Test the reconstruction attack
    best_query_result = query(challenge_id, np.sign(reconstructed_x), submit=True)
    accuracy = (1 + best_query_result / n) / 2
    print(f"Reconstruction attack achieves fraction {accuracy} correct values")

Reconstructed x: [ 1.  1. -1. -1. -1.  1.  1.  1.  1. -1. -1. -1.  1.  1.  1.  1. -1. -1.
 -1.  1.  1. -1. -1.  1.  1. -1. -1. -1.  1. -1. -1.  1.  1. -1.  1. -1.
 -1.  1.  1.  1.  1.  1. -1. -1. -1. -1.  1.  1.  1. -1.  1. -1.  1. -1.
  1.  1.  1. -1.  1. -1. -1.  1. -1.  1.  1.  1.  1. -1.  1.  1. -1. -1.
  1. -1.  1. -1. -1. -1.  1.  1.  1.  1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1.  1.  1. -1.  1.  1.  1.  1.  1.]
Reconstruction attack achieves fraction 0.8400000000000001 correct values
